# 240D: Spatial Audio
### Homework 4

Render a 5.1 channel wav file to binaural audio using the subject 20 of the CIPIC HRTF database.

In [24]:
from netCDF4 import Dataset
%run AudioFile
from scipy import spatial

In [25]:
# Audiofile
a = AudioFile('sample.wav')

#SOFA Dataset
ds = Dataset('subject_020.sofa', 'r')

In [26]:
#Extract some data
IRs = ds['Data.IR']
src_positions = ds['SourcePosition']

#Makes it easier to search
src_positions_tree = spatial.KDTree(src_positions)

In [27]:
def get_IR(azimuth, elevation, distance = 1.0):
    if (azimuth >= 360 or azimuth < 0):
        print ("Invalid azimuth")
        return
    elif (abs(elevation)>=90):
        print ("Invalid elevation")
        return 
    vec = np.array([azimuth, elevation, distance])
    index = src_positions_tree.query(vec)[1]
    return IRs[index]

In [28]:
# 5.1
speaker_layout = np.array([
        [30, 0],  #LF
        [330, 0], #RF
        [0, 0],   #C
        [110, 0], #LR
        [250, 0], #RR
        [0, -30], #LFE       
    ])

In [29]:
ir_list = [get_IR( speaker[0] , speaker[1]) for speaker in speaker_layout]

In [30]:
#Just checking

if( a.sr != ds['Data.SamplingRate'][0] ):
    print "Error! Samplerates dont match"

if( a.nchans != len(ir_list) ):
    print "Number of channels don't match the speaker layout"

In [31]:
# Render to binaural
left = []
right = []

i = 0
for channel in a.audio.T:
    left.append(np.convolve(channel, ir_list[i][0]))
    right.append(np.convolve(channel, ir_list[i][1]))
    i = i +1

left = np.sum(left, axis=0) * 0.2
right = np.sum(right,axis=0)* 0.2    

audio_data = np.array([left, right]).T

In [32]:
# Output to file
output_file = Sndfile('binaural_render.wav' , 'w', Format('wav'), 2, a.sr )
output_file.write_frames(audio_data)

---------

    Akshay Cadambi (akshay19.92@gmail.com)
    Graduate Student
    Media, Arts and Technology
    UC Santa Barbara